# NB! This,InteractiveLoginAuthentication, is only needed to run 1st time, then when ws_config is written, use later CELL in notebook, that just reads that file

In [ ]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/esml/common/")
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication
from esml import ESMLProject

p = ESMLProject()
auth = InteractiveLoginAuthentication(tenant_id = p.tenant)
ws, config_name = p.authenticate_workspace_and_write_config(auth)
p.ws = ws

## 2nd, 3rd this - run thic CELL below. 
- To attach ESML controlplane to your project
- To point at `template-data` for the pipelinbe to know the schema of data
- To init the ESMLPieplinefactory

In [1]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
from baselayer_azure_ml_pipeline import ESMLPipelineFactory, esml_pipeline_types

p = ESMLProject()
p.inference_mode = True
p.active_model = 10 # 10=titanic , 11=Diabetes
p_factory = ESMLPipelineFactory(p)

scoring_date = '1000-01-01 10:35:01.243860' # 
p_factory.batch_pipeline_parameters[1].default_value = scoring_date # overrides ESMLProject.date_scoring_folder.
p_factory.describe()


Using lake_settings.json with ESML version 1.4 - Models array support including LABEL

 ---- Q: WHICH files are generated as templates, for you to EDIT? ---- 
A: These files & locations:
File to EDIT (step: IN_2_SILVER_1): ../../../2_A_aml_pipeline/4_inference/batch/M10/in2silver_ds01_titanic.py
File to EDIT (step: IN_2_SILVER_2): ../../../2_A_aml_pipeline/4_inference/batch/M10/in2silver_ds02_haircolor.py
File to EDIT (step: IN_2_SILVER_3): ../../../2_A_aml_pipeline/4_inference/batch/M10/in2silver_ds03_housing.py
File to EDIT (step: IN_2_SILVER_4): ../../../2_A_aml_pipeline/4_inference/batch/M10/in2silver_ds04_lightsaber.py
File to EDIT (step: SILVER_MERGED_2_GOLD): ../../../2_A_aml_pipeline/4_inference/batch/M10/silver_merged_2_gold.py
File to EDIT (step: SCORING_GOLD): ../../../2_A_aml_pipeline/4_inference/batch/M10/scoring_gold.py
File to EDIT (step: TRAIN_SPLIT_AND_REGISTER): ../../../2_A_aml_pipeline/4_inference/batch/M10/train_split_and_register.py
File to EDIT (step: TRAIN_MANUA

# IN_2_GOLD_SCORING
- Full ML-workflow: If you want to refine data from IN to GOLD, and SCORE model on GOLD, saves SCORED_GOLD in datalake
- Scenario: You want MLOps and full automation, ESMLPipelineFactory starting from Azure Data factory, and calling this genereated Azure ML Pipeline. 
    - Pipeline saving data automatically using the enterprise datalake/ESML AutoLake and ESML SDK

In [ ]:
## BUILD
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=True) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD_SCORING)

In [ ]:
## RUN
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

In [ ]:
# PUBLISH
published_pipeline, endpoint = p_factory.publish_pipeline(batch_pipeline,"_1") # "_1" is optional    to create a NEW pipeline with 0 history, not ADD version to existing pipe & endpoint

In [ ]:
print("In AZURE DATA FACTORY - This is the ID you need, if using PRIVATE LINK, private Azure ML workspace.")
print("-You need PIPELINE id, not pipeline ENDPOINT ID ( since cannot be chosen in Azure data factory if private Azure ML)")
published_pipeline.id

# IN_2_GOLD  only (for scoring/inference purpose)
- If just wanting to refine data to GOLD, before SCORE step
- Scenario: You want to refine data from "IN_2_GOLD" with an easy way to READ/WRITE data (using the enterprise datalake via ESML AutoLake and ESML SDK)

In [ ]:
## BUILD
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=True) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD) # Creates pipeline from template

In [ ]:
## RUN
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

In [ ]:
# PUBLISH
published_pipeline, endpoint = p_factory.publish_pipeline(batch_pipeline) # "_1" is optional    to create a NEW pipeline with 0 history, not ADD version to existing pipe & endpoint

# IN_2_GOLD only - TRAIN...or refined data for Power BI
- If just wanting to refine data to GOLD, before TRAIN-step, `or just to prep data for a Power BI report (No ML involved)`
- Scenario: You want to refine data from "IN_2_GOLD" with an easy way to READ/WRITE data (using the enterprise datalake via ESML AutoLake and ESML SDK)

In [ ]:
p.inference_mode = True
## BUILD
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=False) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD) # Creates pipeline from template

## RUN
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

# Change environment 
- Other curated
- Non-curated, CONDA AutoML
- https://docs.microsoft.com/en-us/azure/machine-learning/resource-curated-environments

In [ ]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
from baselayer_azure_ml_pipeline import ESMLPipelineFactory, esml_pipeline_types

p = ESMLProject()
p.inference_mode = True
p.active_model = 11 # Diabetes
p_factory = ESMLPipelineFactory(p)

scoring_date = '1000-01-01 10:35:01.243860' # 
p_factory.batch_pipeline_parameters[1].default_value = scoring_date # overrides ESMLProject.date_scoring_folder.
p_factory.describe()

## Other curated - change environment

In [ ]:

p_factory.use_curated_automl_environment = True
p_factory.environment_name = "AzureML-AutoML-DNN" # Training[ "AzureML-AutoML", "AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu"]  Inference["AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference",]

## BUILD
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=False) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(pipeline_type=esml_pipeline_types.IN_2_GOLD_SCORING,same_compute_for_all=True)

## Non-curated - custom conda/pip definition

In [ ]:
p_factory.use_curated_automl_environment = False

######### See here for environments: https://docs.microsoft.com/en-us/azure/machine-learning/resource-curated-environments

######### ESML Defaults to the below CONDA, when use_curated_automl_environment = False 
#aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
    #conda_packages=['pandas','scikit-learn'], 
    #pip_packages=['azureml-sdk[automl]', 'pyarrow'])

## BUILD
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=False) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(pipeline_type=esml_pipeline_types.IN_2_GOLD_SCORING,same_compute_for_all=True)

In [ ]:
# RUN
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

# Edit/Customize the ESML auto-generated pipeline

In [29]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
from baselayer_azure_ml_pipeline import ESMLPipelineFactory, esml_pipeline_types
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep

p = ESMLProject()
p.inference_mode = True
p.active_model = 11 # Diabetes
p_factory = ESMLPipelineFactory(p)
scoring_date = '1000-01-01 10:35:01.243860' # 
p_factory.batch_pipeline_parameters[1].default_value = scoring_date # overrides ESMLProject.date_scoring_folder.
#p.ws = p.get_workspace_from_config() 

p_factory.create_dataset_scripts_from_template(overwrite_if_exists=True) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD_SCORING)


#1) Get the ESML auto-generated steps
step_array = p_factory.pipeline_steps_array

for step in step_array:
    t1 = type(step) # https://docs.microsoft.com/en-us/python/api/azureml-pipeline-steps/azureml.pipeline.steps.python_script_step.pythonscriptstep?view=azure-ml-py
    print(str(t1))
    print(step.name)

#2)  EDIT pipeline as you wish...change compute, environments, add steps, remove steps, etc.

pipeline = Pipeline(workspace = p.ws, steps=step_array) # 3) Create a pipelin
print(pipeline)

#4) RUN the pipeline as below:

#pipeline_run = p_factory.execute_pipeline(pipeline)
#pipeline_run.wait_for_completion(show_output=False)

Using lake_settings.json with ESML version 1.4 - Models array support including LABEL
Creates template step_files.py for user to edit at:
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M11/in2silver_ds01_diabetes.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M11/in2silver_ds02_other.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M11/silver_merged_2_gold.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M11/scoring_gold.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M11/train_split_and_register.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M11/train_manual.py
Edit at ../../../2_A_aml_pipeline/4_inference/batch/M11/your_code/your_custom_code.py
Using GEN2 as Datastore
use_project_sp_2_mount: True
ESML will auto-create a compute...
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Using a model specific cluster,

# GOLD_SCORING
- If just wanting to SCORE, with best trained model in Azure ML Studio
- Scenario: Another service, Databricks or Azyre Synapse, refined data fron IN_2_GOLD", you just want to use Azure ML for scoring
    - Azure ML benefits:
        - Get lineage via Azure ML Datasets and Azure ML Model linage. 
        - Host the batch scoring pipleine in your Azure ML Studio
    - EMSL benefits
        - ESMLPipelinefactory + ESML AutoLake design, ESML extra logging & lineage.



In [ ]:
## BUILD
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=False) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT
batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.GOLD_SCORING) # Creates pipeline from template

## RUN
pipeline_run = p_factory.execute_pipeline(batch_pipeline)
pipeline_run.wait_for_completion(show_output=False)

## 1b) When satisfied - `PUBLISH` pipeline (or rebuild and publish)

In [30]:
# PUBLISH
published_pipeline, endpoint = p_factory.publish_pipeline(batch_pipeline, "_1") # "_1" is optional    to create a NEW pipeline with 0 history, not ADD version to existing pipe & endpoint

Created step IN 2 SILVER - ds01_diabetes [e949f734][d27f96d1-9a05-4112-baca-933578ddb4e7], (This step is eligible to reuse a previous run's output)
Created step IN 2 SILVER - ds02_other [5a5aeff1][fdbc753c-0588-4dd2-a558-85f657f6dbea], (This step is eligible to reuse a previous run's output)
Created step SILVER MERGED 2 GOLD [805c9c01][a628abdb-95db-4f02-ae7f-d60658c593ab], (This step is eligible to reuse a previous run's output)
Created step SCORING GOLD [4207f138][9749d505-9b31-4b94-8342-c60ac7053e48], (This step is eligible to reuse a previous run's output)
pub_pipe.name 10_titanic_model_clas_pipe_IN_2_GOLD_SCORING_EP_1
pub_pipe.id 6c3c51e9-df26-4373-ae37-0a31a3616ec0


### 1c) `REBUILD` and `REPUBLISH` on `SAME endpoint` but new version

In [32]:
# REBUILD - if you haven't runned the above cell, uncommen below:
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=False) # overwrite_if_exists=False is default
batch_pipeline = p_factory.create_batch_pipeline(pipeline_type=esml_pipeline_types.IN_2_GOLD_SCORING, same_compute_for_all=True, cpu_gpu_databricks="cpu", allow_reuse=False)  # Gets workspace, connects to lake, creates pipeline.

# PUBLISH
published_pipeline, endpoint = p_factory.publish_pipeline(batch_pipeline, "_6") #  ADD version to existing pipeline & endpoint

Did NOT overwrite script-files with template-files such as 'scoring_gold.py', since overwrite_if_exists=False
Using GEN2 as Datastore
use_project_sp_2_mount: True
ESML will auto-create a compute...
Using a model specific cluster, per configuration in project specific settings, (the integer of 'model_number' is the base for the name)
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Found existing cluster p002-m11weu-dev for project and environment, using it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
image_build_compute = p002-m11weu-dev
Reusing existing compute...
create_gold_to_score_step: inference_mode=True
Created step IN 2 SILVER - ds01_diabetes [de822f4f][5a850ba6-ab47-4f1e-81a1-fb76c82eda75], (This step will run and generate new outputs)
Created step IN 2 SILVER - ds02_other [8d17c

In [35]:
print("In AZURE DATA FACTORY - This is the ID you need, if using PRIVATE LINK, private Azure ML workspace.")
print("-You need PIPELINE id, not pipeline ENDPOINT ID ( since cannot be chosen in Azure data factory if private Azure ML)")
published_pipeline.id

In AZURE DATA FACTORY - This is the ID you need, if using PRIVATE LINK, private Azure ML workspace.
-You need PIPELINE id, not pipeline ENDPOINT ID ( since cannot be chosen in Azure data factory if private Azure ML)


'7fdef475-f619-4f40-8ca1-0001a0166db2'

# 2) `CONSUME` pipeline: HowTo

## 2a) Consume from `Azure Data factory - BATCH_SCORE Pipeline activity`

In [33]:
print("2) Fetch scored data: Below needed for Azure Data factory PIPELINE activity (Pipeline OR Endpoint. Choose the latter") 
print("- PIPELINE ID:  {}".format(published_pipeline.id))
print ("- Endpoint ID")
print("Endpoint ID:  {}".format(endpoint.id))
print("Endpoint Name:  {}".format(endpoint.name))
print("Experiment name:  {}".format(p_factory.experiment_name))

2) Fetch scored data: Below needed for Azure Data factory PIPELINE activity (Pipeline OR Endpoint. Choose the latter
- Endpoint ID
Endpoint ID:  01274209-95e8-4ffb-9b95-9bfb1c865b28
Endpoint Name:  11_diabetes_model_reg_pipe_IN_2_GOLD_SCORING_EP_6
Experiment name:  11_diabetes_model_reg_pipe_IN_2_GOLD_SCORING


## 2a) Get MetaData about scored data: `gold_scored_runinfo` Dataset

In [36]:
from azureml.core.dataset import Dataset
from azureml.core import Experiment
from azureml.pipeline.core import PipelineRun

# 1st you need a "Post scoring" activity, to get metadata of "scored_gold_path" from "last_gold_run.csv"
ds1 = Dataset.get_by_name(workspace = p.ws, name =  p.dataset_gold_scored_runinfo_name_azure)
run_id = ds1.to_pandas_dataframe().iloc[0]["pipeline_run_id"] # ['pipeline_run_id', 'scored_gold_path', 'date_in_parameter', 'date_at_pipeline_run','model_version'])
scored_gold_path = ds1.to_pandas_dataframe().iloc[0]["scored_gold_path"]

print("Read this meta-dataset from ADF: {}/last_gold_run.csv".format(p.path_inference_gold_scored_runinfo))
print("- To get the column 'scored_gold_path' which points to the scored-data:")
print("The scored data will be stored as below, except DateFolders, that will be dynamic, not /1000/01/01")
print("")
print("{}*.parquet".format(scored_gold_path))


Read this meta-dataset from ADF: projects/project002/11_diabetes_model_reg/inference/active/gold_scored_runinfo/last_gold_run.csv
- To get the column 'scored_gold_path' which points to the scored-data:
projects/project002/11_diabetes_model_reg/inference/1/scored/dev/1000/01/01/e2635d17-4812-46e8-9e2b-4178bd4b9215/*.parquet


## 2b) Consume from `from PYTHON`
- Run a pipeline endpoint (`Python SDK` call)

In [38]:
from azureml.pipeline.core import PipelineEndpoint
pipeline_endpoint = PipelineEndpoint.get(workspace=p.ws, name=p_factory.name_batch_pipeline_endpoint +"_6")
pipeline_run_sdk = pipeline_endpoint.submit(p_factory.experiment_name)
pipeline_run_sdk.id

'17d33ab3-aa02-40fe-be8c-8a94f96fa36b'

In [44]:
pipeline_run_sdk.status

'Preparing'

## 2c) Consume from `from REST / PYTHON`
- Run via REST call

In [ ]:
from azureml.pipeline.core import PublishedPipeline,PipelineEndpoint,PipelineRun
import requests
from azureml.core.authentication import ServicePrincipalAuthentication # InteractiveLoginAuthentication, AzureCliAuthentication

sp = p.get_authenticaion_header_sp()
auth_header = sp.get_authentication_header()
date_folder = str(p.date_scoring_folder)
pipeline_endpoint = PipelineEndpoint.get(workspace=p.ws, name=p_factory.name_batch_pipeline_endpoint)

response = requests.post(pipeline_endpoint.endpoint,
                         headers=auth_header,
                         json={"ExperimentName": p_factory.experiment_name,
                               "ParameterAssignments": {
                                     "esml_inference_model_version": p.inferenceModelVersion,
                                     "esml_scoring_folder_date": date_folder
                                     }
                              })

In [ ]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(pipeline_endpoint.endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

### View status from REST call, via SDK

In [ ]:
from azureml.pipeline.core import PublishedPipeline,PipelineEndpoint,PipelineRun
published_pipeline_run = PipelineRun(p.ws.experiments[p_factory.experiment_name], run_id)

In [ ]:
published_pipeline_run.status

# `WHO is the caller, usually?` (Azure Data factory, Azure Devops)` - that sends PARAMETERS and WHY?` 
### Q: Why? 
- A: To use same DEV scoring pipeline, with either different data to be scored `daily scoring`, or `different model-version SAME day` to score with.
- A: To have "environment parameters (dev,test,prod) we can instatiate a ESMLProject what knows the lake, workspace, makes it easy to create 3 pipelines for dev,test,prod
    - And data, if 1 LAKE or 3 LAKES (dev,test,prod), they all have data-folders "dev,test,prod"

### Who gives input?
- A) Azure Devops (CI/CD) will trigger TRAIN pipeline, that will end with creating this BATCH SCORING, with 
    - 2 parameters (`esml_environment, esml_inference_model_version`), to CREATE/UPDATE the BATCH pipeline with newly trained model
    - 1 dummy (`esml_scoring_folder_date`) to test BATCH SCORING after creation.
- B) Azure Datafactory (read from source, writes as .csv or .parquet to IN-folder), and will trigger BATCH SCORING with:
    - 2 PIPELINE parameters (`esml_inference_model_version, esml_scoring_folder_date`), to read IN-DATA to be scored. Usually "todays" esml_scoring_folder_date
    - Note: To solve "many scorings same day", a "run.id" folder is created before the actual data.parquet
    - Note: `*esml_environment` is not really needed post creation - since we already created the pipleine in DEV, `locked and loaded`

### Who needs `scored_data` and HOW to get it? META data:
- `Azure Datafactory` can read meta data of `last scored GOLD`, to get datalake-path of SCORED_GOLD - can then "`write back scored data`" to source, or another `system`
    - See next cells "`Get previous RUN and PIPELINE via `ESML` metadata`"
- `Power BI` can read the meta-data to fetch `last scored GOLD` directly

## Get previous RUN and PIPELINE via `ESML` metadata
- How to get path of `scored_gold_path` and how to see the actual `pipeline run`

In [41]:
from azureml.core.dataset import Dataset
from azureml.core import Experiment
from  azureml.pipeline.core import PipelineRun

# Get "Pipeline run" info, for tghe most recent "latest scored gold"
ds1 = Dataset.get_by_name(workspace = p.ws, name =  p.dataset_gold_scored_runinfo_name_azure)
run_id = ds1.to_pandas_dataframe().iloc[0]["pipeline_run_id"] # ['pipeline_run_id', 'scored_gold_path', 'date_in_parameter', 'date_at_pipeline_run','model_version'])
scored_gold_path = ds1.to_pandas_dataframe().iloc[0]["scored_gold_path"]

print("dataset_gold_scored_runinfo, location: {}".format)
print("pipeline_run_id: {}".format(run_id))
print("scored_gold_path: '{}'".format(scored_gold_path))

experiment = Experiment(workspace=p.ws, name=p_factory.experiment_name)
remote_run = PipelineRun(experiment=experiment, run_id=run_id)
print("\nFetched RUN object {}".format(remote_run))

<built-in method format of str object at 0x0000018B93D9CB10>
pipeline_run_id: e2635d17-4812-46e8-9e2b-4178bd4b9215
scored_gold_path: 'projects/project002/11_diabetes_model_reg/inference/1/scored/dev/1000/01/01/e2635d17-4812-46e8-9e2b-4178bd4b9215/'

Fetched RUN object Run(Experiment: 11_diabetes_model_reg_pipe_IN_2_GOLD_SCORING,
Id: e2635d17-4812-46e8-9e2b-4178bd4b9215,
Type: azureml.PipelineRun,
Status: Completed)


# `What can you configure?` (parameters, step compute, custom code)
## 1) Configure: Parameters
- Pipeline parameters: scoring_date, model_version
    - Why: To dynamically select different data & model to score with, with same pipeline/reuse.
    - Who: Azure data factory can dynamically set these, and call AML pipline
- Pipeline parameters (model specific): target_column_name
    - Why: To merge datasets to GOLD.
print("Model version (pipeline parameter): {}".format(p_factory.batch_pipeline_parameters[0].default_value))
print(" - This default value is set from ESMLProject settings: {}".format(p.inferenceModelVersion))
print("Scoring datetime: {}".format(p_factory.batch_pipeline_parameters[1].default_value))
print(" - This default value is set from ESMLProject settings: {}".format(str(p.date_scoring_folder)))
# Optional parameters to READ or SET
#parameters[2].name: parameters[2].default_value, # esml_optional_unique_scoring_folder 
#parameters[3].name: parameters[3].default_value # par_esml_dev_test_prod
## 2) Configure: Compute & Environment (via ESML config or inject your own)
- `Different compute per step OR samee for all` ["cpu","gpu", "databricks"], based on your ESML environment (dev,test,prod) compute settings, and Dataset properties.
 - A) `Different compute for all steps`

        -  if(d.cpu_gpu_databricks == "cpu"):
        -       compute, runconfig = self.init_cpu_environment()
        -  elif(d.cpu_gpu_databricks == "databricks"):
        -     compute, runconfig = self.init_databricks_environment()
        -  elif(d.cpu_gpu_databricks == "gpu"):
        -       compute, runconfig = self.init_gpu_environment()
- B) `Same compute for all`: For the full pipeline, is the DEFAULT behaviour.

        - def `create_batch_scoring_pipe(self, `same_compute_for_all=True`, `cpu_gpu_databricks="cpu")`
